In [6]:
import pandas as pd
import plotly_express as px

dataframe_mat = 'Data/Mobiliteitstrend__per_rit_en_motief_02102024_122208.csv'
dataframe_wfh1 = 'Data/Werkzame_beroepsbevolking__thuiswerken_04102024_193141.csv'
dataframe_wfh2 ='Data/Werkzame_beroepsbevolking__thuiswerken_07102024_101921.csv'

df_mat = pd.read_csv(dataframe_mat,  delimiter=";", encoding="UTF-8-SIG")
df_mat['Perioden'] = df_mat['Perioden'].str.replace('*', '', regex=False)

df_mat_total = df_mat[
    (df_mat['Geslacht'] == "Totaal mannen en vrouwen") &
    (df_mat["Leeftijd"] == "Totaal") &
    (df_mat["Vervoerwijzen"] != "Totaal") &
    (df_mat["Reismotieven"] == "Van en naar het werk")
]


df_wfh1 = pd.read_csv(dataframe_wfh1, delimiter=";", encoding="UTF-8-SIG")
df_wfh2 = pd.read_csv(dataframe_wfh2, delimiter=";", encoding="UTF-8-SIG")


df_wfh2.rename(columns={
    'Positie in de werkkring': 'Positie werkkring',
    'Thuiswerken': 'Thuiswerker'
}, inplace=True)

# Merge WFH datasets
df_merged = pd.concat([df_wfh1, df_wfh2])

# Map WFH categories
thuiswerker_mapping = {
    'Thuiswerker die gewoonlijk thuis werkt': 'Meestal of soms thuiswerken',
    'Thuiswerker, incidenteel op vaste dagen': 'Meestal of soms thuiswerken',
    'Thuiswerker, incidenteel geen vaste dag': 'Meestal of soms thuiswerken',
    'Geen thuiswerker': 'Niet thuiswerken'
}
df_merged['Thuiswerker'] = df_merged['Thuiswerker'].replace(thuiswerker_mapping)

# Filter WFH data
df_filtered = df_merged[
    (df_merged['Positie werkkring'] == 'Totaal') &
    (df_merged['Persoonskenmerken'] == 'Totaal personen') &
    (~df_merged['Thuiswerker'].isin([
        "Thuiswerker",
        "Totaal",
        "Meestal thuiswerken",
        "Soms thuiswerken",
        "Thuiswerker die incidenteel thuis werkt"
    ]))
]

# Convert Perioden to string type in both dataframes
df_filtered['Perioden'] = df_filtered['Perioden'].astype(str)
df_mat_total['Perioden'] = df_mat_total['Perioden'].astype(str)

# Print unique values to check matching periods
print("Periods in df_filtered:", df_filtered['Perioden'].unique())
print("Periods in df_mat_total:", df_mat_total['Perioden'].unique())

# Merge filtered datasets on 'Perioden'
final_df = pd.merge(df_filtered, df_mat_total, on='Perioden', how='inner')

final_df_agg = final_df.groupby('Perioden').agg({
    'Werkzame beroepsbevolking (x 1 000)': 'sum',
    'Ritten per persoon per dag (gemiddeld) (aantal)': 'first', 
    'Leeftijd': 'first',
    'Vervoerwijzen': 'first',
    'Reismotieven': 'first',
    'Thuiswerker': 'first',
    'Positie werkkring': 'first',
    'Persoonskenmerken': 'first'
}).reset_index()

final_df_agg

Periods in df_filtered: ['2013' '2014' '2015' '2016' '2017' '2018' '2019' '2020' '2021' '2022'
 '2023']
Periods in df_mat_total: ['1999' '2003' '2007' '2011' '2015' '2019' '2020' '2021' '2022' '2023']


C:\Users\Pelle Boucher\AppData\Local\Temp\ipykernel_25120\187803277.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Perioden'] = df_filtered['Perioden'].astype(str)
C:\Users\Pelle Boucher\AppData\Local\Temp\ipykernel_25120\187803277.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mat_total['Perioden'] = df_mat_total['Perioden'].astype(str)


,Perioden,Werkzame beroepsbevolking (x 1 000),Ritten per persoon per dag (gemiddeld) (aantal),Leeftijd,Vervoerwijzen,Reismotieven,Thuiswerker,Positie werkkring,Persoonskenmerken
0,2015,58065,0.25,Totaal,Auto (bestuurder),Van en naar het werk,Meestal of soms thuiswerken,Totaal,Totaal personen
1,2019,62664,0.25,Totaal,Auto (bestuurder),Van en naar het werk,Meestal of soms thuiswerken,Totaal,Totaal personen
2,2020,62657,0.19,Totaal,Auto (bestuurder),Van en naar het werk,Meestal of soms thuiswerken,Totaal,Totaal personen
3,2021,64785,0.19,Totaal,Auto (bestuurder),Van en naar het werk,Meestal of soms thuiswerken,Totaal,Totaal personen
4,2022,66836,0.21,Totaal,Auto (bestuurder),Van en naar het werk,Meestal of soms thuiswerken,Totaal,Totaal personen
5,2023,68159,0.22,Totaal,Auto (bestuurder),Van en naar het werk,Meestal of soms thuiswerken,Totaal,Totaal personen


In [7]:
px.scatter(final_df_agg,
           x="Werkzame beroepsbevolking (x 1 000)",
           y='Ritten per persoon per dag (gemiddeld) (aantal)',
           trendline= "ols"
           )

